In [1]:
# Deep Learning CNN model to recognize face
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''####### IMAGE PRE-PROCESSING for TRAINING and TESTING data #######'''

# Specifying the folder where images are present
TrainingImagePath = 'Face Images/Final Training Images'

from keras.preprocessing.image import ImageDataGenerator
# Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images
train_datagen = ImageDataGenerator(
        shear_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images
test_datagen = ImageDataGenerator()

# Generating the Training Data
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=8,
        class_mode='categorical')


# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64),
        batch_size=8,
        class_mode='categorical')

# Printing class labels for each face
test_set.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [2]:
'''############ Creating lookup table for all faces ############'''
# class_indices have the numeric tag for each face
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag for future reference
ResultMap={}
for faceValue,faceName in zip(TrainClasses.values(),TrainClasses.keys()):
    ResultMap[faceValue]=faceName

# Saving the face map for future reference
import pickle
with open("ResultsMap.pkl", 'wb') as fileWriteStream:
    pickle.dump(ResultMap, fileWriteStream)

# The model will give answer as a numeric tag
# This mapping will help to get the corresponding face name for it
print("Mapping of Face and its ID",ResultMap)

# The number of neurons for the output layer is equal to the number of faces
OutputNeurons=len(ResultMap)
print('\n The Number of output neurons: ', OutputNeurons)

Mapping of Face and its ID {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}

 The Number of output neurons:  16


In [3]:
'''######################## Create CNN deep learning model ########################'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels
'''
classifier.add(Convolution2D(8, kernel_size=(5, 5), strides=(1, 1), input_shape=(64,64,3), activation='relu'))

'''# STEP--2 MAX Pooling'''
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############## ADDITIONAL LAYER of CONVOLUTION for better accuracy #################'''
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))

'''# STEP--3 FLattening'''
classifier.add(Flatten())

'''# STEP--4 Fully Connected Neural Network'''
classifier.add(Dense(64, activation='relu'))

classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# Compiling the CNN'''
#classifier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])

###########################################################
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit(
                    training_set,
                    steps_per_epoch=30,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=10)

EndTime=time.time()
print("###### Total Time Taken: ", round((EndTime-StartTime)/60), 'Minutes ######')

Epoch 1/10
30/30 [==============================] - 3s 71ms/step - loss: 44.0663 - accuracy: 0.1000 - val_loss: 2.1177 - val_accuracy: 0.3625
Epoch 2/10
30/30 [==============================] - 1s 48ms/step - loss: 1.9221 - accuracy: 0.4195 - val_loss: 1.0427 - val_accuracy: 0.6750
Epoch 3/10
30/30 [==============================] - 1s 48ms/step - loss: 1.0605 - accuracy: 0.7034 - val_loss: 0.9347 - val_accuracy: 0.7750
Epoch 4/10
30/30 [==============================] - 1s 48ms/step - loss: 0.8016 - accuracy: 0.7627 - val_loss: 0.4560 - val_accuracy: 0.8750
Epoch 5/10
30/30 [==============================] - 1s 47ms/step - loss: 0.6915 - accuracy: 0.8136 - val_loss: 0.2015 - val_accuracy: 0.9625
Epoch 6/10
30/30 [==============================] - 1s 48ms/step - loss: 0.4294 - accuracy: 0.8814 - val_loss: 0.4611 - val_accuracy: 0.8500
Epoch 7/10
30/30 [==============================] - 1s 48ms/step - loss: 0.4059 - accuracy: 0.8644 - val_loss: 0.2121 - val_accuracy: 0.9125
Epoch 8/10
3

In [4]:
'''########### Making single predictions ###########'''
import numpy as np
from tensorflow.keras.preprocessing import image

ImagePath = 'Face Images/Final Testing Images/face4/3face4.jpg'
test_image = image.load_img(ImagePath,target_size=(64, 64))
test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image,axis=0)

result=classifier.predict(test_image,verbose=0)
print(training_set.class_indices)

print('####'*10)
print('Prediction is: ',ResultMap[np.argmax(result)])

{'face1': 0, 'face10': 1, 'face11': 2, 'face12': 3, 'face13': 4, 'face14': 5, 'face15': 6, 'face16': 7, 'face2': 8, 'face3': 9, 'face4': 10, 'face5': 11, 'face6': 12, 'face7': 13, 'face8': 14, 'face9': 15}
########################################
Prediction is:  face4


In [5]:
import numpy
test_generator = ImageDataGenerator()
test_data_path = 'Face Images/Final Testing Images'
test_data_generator = test_generator.flow_from_directory(
    test_data_path, # Put your path here
     target_size=(64, 64),
    batch_size=8,
    shuffle=False)
test_steps_per_epoch = numpy.math.ceil(test_data_generator.samples / test_data_generator.batch_size)

predictions = classifier.predict(test_data_generator, steps=test_steps_per_epoch)
# Get most likely class
predicted_classes = numpy.argmax(predictions, axis=1)

Found 64 images belonging to 16 classes.
8/8 [==============================] - 0s 35ms/step


In [6]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())  

In [7]:
from sklearn import metrics
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report) 

              precision    recall  f1-score   support

       face1       1.00      0.75      0.86         4
      face10       1.00      1.00      1.00         4
      face11       0.80      1.00      0.89         4
      face12       1.00      0.75      0.86         4
      face13       1.00      0.75      0.86         4
      face14       0.75      0.75      0.75         4
      face15       1.00      1.00      1.00         4
      face16       0.80      1.00      0.89         4
       face2       1.00      0.75      0.86         4
       face3       0.80      1.00      0.89         4
       face4       0.80      1.00      0.89         4
       face5       1.00      1.00      1.00         4
       face6       1.00      1.00      1.00         4
       face7       1.00      1.00      1.00         4
       face8       1.00      0.75      0.86         4
       face9       0.80      1.00      0.89         4

    accuracy                           0.91        64
   macro avg       0.92   

In [45]:
from collections import Counter
Counter(true_classes)


Counter({0: 4,
         1: 4,
         2: 4,
         3: 4,
         4: 4,
         5: 4,
         6: 4,
         7: 4,
         8: 4,
         9: 4,
         10: 4,
         11: 4,
         12: 4,
         13: 4,
         14: 4,
         15: 4})

In [46]:
Counter(predicted_classes)

Counter({0: 3,
         9: 7,
         1: 5,
         2: 4,
         3: 5,
         4: 2,
         12: 6,
         5: 3,
         7: 4,
         13: 5,
         8: 3,
         11: 5,
         10: 3,
         15: 5,
         14: 4})